In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
print(mnist.train.images.shape)

(55000, 784)


In [5]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

EPOCHS = 10
BATCH_SIZE = 128

In [6]:
# dropout rate is set as a placeholder
keep_prob = tf.placeholder(tf.float32)

In [7]:
# input placeholder
X = tf.placeholder(tf.float32,[None,784])
X_img = tf.reshape(X,[-1,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])

In [10]:
# filter size 3x3
# image shape for L1 is (?,28,28,1)
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev = 0.01))
L1 = tf.nn.conv2d(X_img,W1,strides = [1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L1 = tf.nn.dropout(L1,keep_prob=keep_prob)

In [11]:
## L2 image shape =(?,14,14,32)

In [12]:
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev = 0.01))
L2 = tf.nn.conv2d(L1,W2,strides = [1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2,ksize = [1,2,2,1],strides= [1,2,2,1],padding = 'SAME')
L2 = tf.nn.dropout(L2,keep_prob=keep_prob)

In [13]:
## L3 image shape = (?,7,7,64)
W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev = 0.01))
# 3x3 filter size, # of filter 128
L3 = tf.nn.conv2d(L2,W3,strides = [1,1,1,1],padding = 'SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3,ksize = [1,2,2,1],strides= [1,2,2,1],padding = 'SAME')
L3 = tf.nn.dropout(L3,keep_prob = keep_prob)
# flattening  (?,4,4,128)
L3_flat = tf.reshape(L3,[-1,128*4*4])

In [14]:
W4 = tf.get_variable('W4',shape=[128*4*4,625],initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat,W4)+b4)
L4 = tf.nn.dropout(L4,keep_prob=keep_prob)

In [15]:
fc1_W = tf.get_variable('fc1_W',shape = [625,120],initializer = tf.contrib.layers.xavier_initializer())
fc1_b = tf.Variable(tf.random_normal([120]))
fc1 = tf.matmul(L4, fc1_W) + fc1_b

In [16]:
fc1 = tf.nn.relu(fc1)

In [17]:
fc2_W = tf.get_variable('fc2_W',shape = [120,84],initializer = tf.contrib.layers.xavier_initializer())
fc2_b = tf.Variable(tf.random_normal([84]))
fc2 = tf.matmul(fc1,fc2_W)+fc2_b

In [18]:
fc2 = tf.nn.relu(fc2)

In [19]:
fc3_W = tf.get_variable('fc3_W',shape = [84,10], initializer = tf.contrib.layers.xavier_initializer())
fc3_b = tf.Variable(tf.random_normal([10]))
fc3 = tf.matmul(fc2,fc3_W)+fc3_b

In [20]:
logits = fc3

In [21]:
print(logits.shape)
# define cost/loss & optimizer

(?, 10)


In [22]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [23]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [24]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy

# if you have a OOM error, please refer to lab-11-X-mnist_deep_cnn_low_memory.py

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))


Learning started. It takes sometime.
Epoch: 0001 cost = 0.503452519
Epoch: 0002 cost = 0.106380229
Epoch: 0003 cost = 0.082011244
Epoch: 0004 cost = 0.066317661
Epoch: 0005 cost = 0.058758262
Epoch: 0006 cost = 0.052976746
Epoch: 0007 cost = 0.046532387
Epoch: 0008 cost = 0.044784712
Epoch: 0009 cost = 0.043101762
Epoch: 0010 cost = 0.039505894
Epoch: 0011 cost = 0.038042728
Epoch: 0012 cost = 0.036714312
Epoch: 0013 cost = 0.032960032
Epoch: 0014 cost = 0.032043161
Epoch: 0015 cost = 0.032008597
Learning Finished!
Accuracy: 0.9938
Label:  [4]
Prediction:  [4]
